Sales data:  Censored<BR>
Customer list: Censored 

In [1]:
pip install geopy

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3
import urllib.request, urllib.parse
from urllib import request, parse
from bs4 import BeautifulSoup
import json

!gdown Censored -O 'customer address list.xlsx'
df_cust_list=pd.read_excel('customer address list.xlsx')
!gdown Censored -O 'DSBS data text.xlsx'
df_sales=pd.read_excel('DSBS data text.xlsx')


Downloading...
From: Censored
100% 84.3k/84.3k [00:00<00:00, 113MB/s]
Downloading...
From: Censored
100% 9.01M/9.01M [00:00<00:00, 74.9MB/s]


In [3]:
df_cust_list['Full_Address']=df_cust_list['Address'].astype(str)+" "+df_cust_list['Address 2'].astype(str)
df_cust_list['Full_Address']=df_cust_list['Full_Address'].str.replace('nan','',).str.strip()
df_cust_list['Full_Address'].replace('', np.nan, inplace=True)
df_cust_list.dropna(subset=['Full_Address'],inplace=True)

# Initialize the geocoder
geolocator = GoogleV3(api_key='Censored')

# Define a function to geocode an address and return the latitude and longitude
def geocode_address(address):
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the geocode_address function to the '營業地址 / Business Address' column
df_cust_list[['latitude', 'longitude']] = df_cust_list['Full_Address'].apply(geocode_address).apply(pd.Series)


In [4]:
df_empty=df_cust_list[df_cust_list['latitude'].isna()]

In [5]:
df_empty.head()

,Blocked,Bill-to Customer No.,No.,Address,Address 2,County,City,Customer Class,Customer Class Name,Full_Address,latitude,longitude
18,Closed,C000462,C000019,好運中心商場 Level 3 - 40 號舖位,NaN,NaN,NT,C10213,Click&Mortar Pet Stores Chains,好運中心商場 Level 3 - 40 號舖位,NaN,NaN
23,,C000462,C000024,新都城3期2樓 241-242 舖,NaN,NaN,NT,C10213,Click&Mortar Pet Stores Chains,新都城3期2樓 241-242 舖,NaN,NaN
26,,C000462,C000027,時代廣場北翼L2-38號鋪,NaN,NaN,NT,C10213,Click&Mortar Pet Stores Chains,時代廣場北翼L2-38號鋪,NaN,NaN
27,,C000462,C000028,屯門市廣場第一期地下(家品部側),NaN,NaN,NT,C10213,Click&Mortar Pet Stores Chains,屯門市廣場第一期地下(家品部側),NaN,NaN
37,,NaN,C000038,新都城1期地下G69-70號舖,NaN,NaN,NT,C10213,Click&Mortar Pet Stores Chains,新都城1期地下G69-70號舖,NaN,NaN


In [6]:
lat_list=[]
lon_list=[]
n=0
#index_list=[]
try:
  for address in df_empty['Full_Address']:
    service_url='https://www.als.ogcio.gov.hk/lookup?q='
    params = {'q' : address}
    url_get = service_url + "?" + parse.urlencode(params)
    url_get = service_url + parse.urlencode(params)
    data_get = request.urlopen(url_get).read().decode()
    xml= BeautifulSoup(data_get, "xml")
    try:
      lat=xml.find("Latitude").text
      lon=xml.find("Longitude").text
    except:
      service_url='https://www.als.ogcio.gov.hk/lookup?q='
      params = {'q' : df_empty['Address 2'].iloc[n]}
      url_get = service_url + "?" + parse.urlencode(params)
      url_get = service_url + parse.urlencode(params)
      data_get = request.urlopen(url_get).read().decode()
      xml= BeautifulSoup(data_get, "xml")
      lat=xml.find("Latitude").text
      lon=xml.find("Longitude").text
    lat_list.append(lat)
    lon_list.append(lon)
    n+=1
except:
  print(n)

#df_empty[['latitude', 'longitude']] = [lat_list, lon_list]
df_empty['latitude'] = lat_list
df_empty['longitude'] = lon_list

<ipython-input-6-3d1dc2d34054>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty['latitude'] = lat_list
<ipython-input-6-3d1dc2d34054>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_empty['longitude'] = lon_list


In [7]:
df_test=df_cust_list
df_test=df_test.merge(df_empty[['No.','latitude','longitude']],on='No.',how='left')
df_test['latitude_x']=df_test['latitude_x'].fillna(0)
df_test['longitude_x']=df_test['longitude_x'].fillna(0)
df_test['latitude_y']=df_test['latitude_y'].fillna(0)
df_test['longitude_y']=df_test['longitude_y'].fillna(0)
df_test['latitude_x']=df_test['latitude_x'].astype(float)
df_test['latitude_y']=df_test['latitude_y'].astype(float)
df_test['longitude_x']=df_test['longitude_x'].astype(float)
df_test['longitude_y']=df_test['longitude_y'].astype(float)
df_test['latitude']=df_test['latitude_x']+df_test['latitude_y']
df_test['longitude']=df_test['longitude_x']+df_test['longitude_y']
df_test = df_test.drop(columns=['latitude_x', 'latitude_y','longitude_x','longitude_y'])

In [8]:
country_list=[]
c=0
try:
  for x,y in zip(df_test['longitude'],df_test['latitude']):
    service_url='https://maps.googleapis.com/maps/api/geocode/json?'
    params = {'latlng' : str(y)+','+str(x)}
    suffix='&key=Censored'

    url_get = service_url + parse.urlencode(params) + suffix
    url_get=parse.unquote(url_get)
    data_get = request.urlopen(url_get).read().decode()
    js = json.loads(data_get)
    index=-1
    while js['results'][-1]['address_components'][index]['types']!=['country', 'political']:
      index-=1
    country=js['results'][-1]['address_components'][index]['long_name']
    country_list.append(country)
    c+=1
except:
  print(c)
df_test['County']=country_list

In [9]:
df_HK=df_test.query("County == 'Hong Kong'" )

In [18]:
distric_list = []
d=0
try:
  for lon,lat in zip(df_HK['longitude'],df_HK['latitude']):

    service_url='https://api.csdi.gov.hk/apim/dataquery/api/?id=had_rcd_1634523272907_75218&layer=dcd&'
    params = {'bbox-crs=WGS84&bbox' : str(lon)+','+str(lat)+','+str(lon)+','+str(lat)}
    suffix='&limit=1&offset=0'

    url_get = service_url + parse.urlencode(params) + suffix
    url_get=parse.unquote(url_get)
    data_get = request.urlopen(url_get).read().decode()
    js = json.loads(data_get)
    if len(js['features'][0]['properties']['NAME_EN'])>0:
      distric=js['features'][0]['properties']['NAME_EN']
    else:
      distric='error'
    distric_list.append(distric)
    d+=1
except:
  print(d)

df_HK['distric']=distric_list
df_HK=df_HK.reset_index(drop=True)

<ipython-input-18-aa769a45da9e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HK['distric']=distric_list


In [26]:
d_list=df_HK['distric']
d_list.drop_duplicates()

0                  Sha Tin District
1                Kwun Tong District
2                    North District
4                 Southern District
5                Wan Chai District 
6                  Eastern District
9             Kowloon City District
10              Kwai Tsing District
12            Wong Tai Sin District
13           Yau Tsim Mong District
15                Sai Kung District
16     Central and Western District
20                  Tai Po District
24               Tsuen Wan District
26                Tuen Mun District
28               Yuen Long District
67            Sham Shui Po District
150                Islands District
Name: distric, dtype: object

In [27]:
df_sales.head()

,Channel,Bill-To Customer,Ship-to Customer,Animal Species,Life Stage Grouping,Life Stage,Animal Size,Year,Quarter,Month,Pets Fed
0,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,Growth,Junior,Maxi,2019,Q4,October,1.185958
1,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,Growth,Junior,Maxi,2019,Q4,November,2.450980
2,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,Adult,Adult,Mini,2019,Q4,December,5.259467
3,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,Adult,Adult,Maxi,2019,Q4,October,8.488964
4,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,Adult,Adult,Maxi,2019,Q4,November,21.147463


In [84]:
df_fed=df_sales[['Channel','Bill-To Customer','Ship-to Customer','Animal Species','Year','Month','Pets Fed']]
df_fed=df_fed.groupby(['Channel','Ship-to Customer','Animal Species','Year','Month'])['Pets Fed'].sum().reset_index()
df_fed=df_fed.query("Channel!='OTHER'").reset_index(drop=True)
df_fed['No.']=df_fed['Ship-to Customer'].str.split("(", expand=True)[0].str.strip()
df_fed

,Channel,Ship-to Customer,Animal Species,Year,Month,Pets Fed,No.
0,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2019,December,59.022908,C000231
1,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2019,November,23.598444,C000231
2,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2019,October,9.674923,C000231
3,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2020,April,50.261016,C000231
4,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2020,August,35.856949,C000231
...,...,...,...,...,...,...,...
41272,VET,S100345 (HEARTIEST VETERINARY MEDICAL CENTRE),Dog,2023,August,3.541360,S100345
41273,VET,S100345 (HEARTIEST VETERINARY MEDICAL CENTRE),Dog,2023,July,5.597587,S100345
41274,VET,S100345 (HEARTIEST VETERINARY MEDICAL CENTRE),Dog,2023,September,21.026700,S100345
41275,VET,S100351 (BONDING PET AND EXOTIC ANIMAL HOSPITAL),Cat,2023,September,11.763340,S100351


In [86]:
df_fed.merge(df_HK[['No.','distric']],how='left',left_on='No.',right_on="No.")

,Channel,Ship-to Customer,Animal Species,Year,Month,Pets Fed,No.,distric
0,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2019,December,59.022908,C000231,North District
1,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2019,November,23.598444,C000231,North District
2,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2019,October,9.674923,C000231,North District
3,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2020,April,50.261016,C000231,North District
4,PRO,C000231 (HONG KONG SEEING EYE DOG SERVICES),Dog,2020,August,35.856949,C000231,North District
...,...,...,...,...,...,...,...,...
41272,VET,S100345 (HEARTIEST VETERINARY MEDICAL CENTRE),Dog,2023,August,3.541360,S100345,Yuen Long District
41273,VET,S100345 (HEARTIEST VETERINARY MEDICAL CENTRE),Dog,2023,July,5.597587,S100345,Yuen Long District
41274,VET,S100345 (HEARTIEST VETERINARY MEDICAL CENTRE),Dog,2023,September,21.026700,S100345,Yuen Long District
41275,VET,S100351 (BONDING PET AND EXOTIC ANIMAL HOSPITAL),Cat,2023,September,11.763340,S100351,Kwai Tsing District


1. distric vs. VET/SPT count
2. distric vs. dog/cat time series-->2 plot 18 line per species
3. growth rate on cat/dog & distric per time -->table
